In [1]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 13.0.1+9, mixed mode, sharing)
  Starting server from c:\users\vishw\appdata\local\programs\python\python36\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\vishw\AppData\Local\Temp\tmprm8_xh1t
  JVM stdout: C:\Users\vishw\AppData\Local\Temp\tmprm8_xh1t\h2o_vishw_started_from_python.out
  JVM stderr: C:\Users\vishw\AppData\Local\Temp\tmprm8_xh1t\h2o_vishw_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.1
H2O_cluster_version_age:,2 months and 1 day
H2O_cluster_name:,H2O_from_python_vishw_5t3zys
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.945 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [2]:
data_path="https://github.com/h2oai/h2o-tutorials/raw/master/h2o-world-2017/automl/data/powerplant_output.csv"

# Load data into H2O
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
df.describe()

Rows:9568
Cols:5




,TemperatureCelcius,ExhaustVacuumHg,AmbientPressureMillibar,RelativeHumidity,HourlyEnergyOutputMW
type,real,real,real,real,real
mins,1.81,25.36,992.89,25.56,420.26
mean,19.651231187290968,54.30580372073578,1013.2590781772575,73.30897784280936,454.36500940635455
maxs,37.11,81.56,1033.3,100.16,495.76
sigma,7.452473229611079,12.707892998326809,5.938783705811604,14.600268756728953,17.066994999803416
zeros,0,0,0,0,0
missing,0,0,0,0,0
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.4,1012.16,92.14,488.56


In [4]:
y = "HourlyEnergyOutputMW"

In [5]:
splits = df.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

In [6]:
aml = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "powerplant_lb_frame")
aml.train(y = y, training_frame = train, leaderboard_frame = test)

AutoML progress: |
21:16:19.293: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [7]:
aml.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_4_AutoML_20200604_211619,10.4884,3.23858,10.4884,2.24681,0.00713384
GBM_3_AutoML_20200604_211619,11.1887,3.34496,11.1887,2.35355,0.00736291
GBM_2_AutoML_20200604_211619,11.7178,3.42313,11.7178,2.43236,0.00753936
GBM_grid__1_AutoML_20200604_211619_model_5,11.9668,3.45931,11.9668,2.42902,0.00762467
GBM_grid__1_AutoML_20200604_211619_model_4,12.2086,3.49409,12.2086,2.46433,0.00769577
XRT_1_AutoML_20200604_211619,12.6359,3.5547,12.6359,2.51276,0.0078428
GBM_grid__1_AutoML_20200604_211619_model_6,12.8486,3.58449,12.8486,2.60439,0.00789145
GBM_grid__1_AutoML_20200604_211619_model_2,12.8746,3.58811,12.8746,2.60445,0.00790374
GBM_1_AutoML_20200604_211619,13.0668,3.6148,13.0668,2.58808,0.00796518
GBM_5_AutoML_20200604_211619,13.5758,3.68453,13.5758,2.67988,0.00809465


In [10]:
pred = aml.predict(test)
pred.head()

gbm prediction progress: |████████████████████████████████████████████████| 100%


predict
486.385
473.862
466.246
452.421
447.859
469.419
442.53
464.351
442.865
431.683


In [11]:
perf = aml.leader.model_performance(test)
perf


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 10.488406266906058
RMSE: 3.238580903251617
MAE: 2.2468129614124814
RMSLE: 0.007133840642635012
Mean Residual Deviance: 10.488406266906058
